# <font color = 'green'>GBM(Gradient Boosting Machine) p.222~

## ✔ GBM의 개요 및 실습

- **부스팅 알고리즘** 
    - **여러 개의 약한 학습기를 순차적으로 학습-예측**하면서 **잘못 예측한 데이터에 가중치 부여**를 통해 **오류를 개선해 나가면서 학습하는 방식**
   
   
   
- 부스팅 알고리즘의 대표적인 구현
    - **AdaBoost(에이다 부스트)** : **오류 데이터에 가중치를 부여하면서 부스팅을 수행**하는 대표적인 알고리즘
    
    
- GBM은 과적합에도 강한 **뛰어난 예측성능**을 가지고 있으나 **수행 시간이 오래걸린다**는 단점이 있음
    

### 에이다부스트

![title](에이다부스트.jpg)

- +와 -로 구성된 피처 데이터 세트가 있다고 가정


- ***Step 1***
    - 첫 번째 약한 학습기가 분류 기준 1로 +와 -를 예측한 것.
    - ⚪가 표시된 데이터는 잘못 분류된 **오류 데이터**


- ***Step 2*** 
    - 오류 데이터에 대해서 **가중치 값 부여**
    - 가중치가 부여된 오류 데이터는 다음 약한 학습기가 더 잘 분류할 수 있도록 해준다(크기가 커짐)
    
    
- ***Step 3***
    - 두 번째 약한 학습기가 분류 기준 2로 데이터 예측
    - ⚪가 표시된 데이터는 잘못 분류된 **오류 데이터**


- ***Step 4***
    - 잘못 분류된 오류 데이터에 더 큰 가중치 부여
    
    
- ***Step 5***
    - 세 번재 약한 학습기가 분류 기준 3으로 +와 -를 분류하고 오류 데이터를 찾는다.
    - 약한 학습기가 순차적으로 오류 값에 대해 가중치를 부여한 예측 결정 기준을 모두 결합해 예측을 수행한다.
    
    
- 맨 아래는 첫 번재, 두 번재, 세 번째 약한 학습기를 모두 결합한 결과 예측

![title](에이다부스트2.jpg)

- 개별 약한 학습기는 그림과 같이 **각각 가중치를 부여해 결합**한다.

- 첫 번째 학습기에 가중치 0.3, 두 번째 학습기에 가중치 0.5, 세 번째 학습기에 가중치 0.8을 부여한 후 **모두 결합해 예측을 수행**한다.

### GBM(Gradient Boost Machine)

- 에이다부스트와 달리 가중치 업데이트를 **경사하강법**을 이용함.


- **<font color = 'Navy'>경사 하강법(Gradient Descent)**
    - **오류식을 최소화하는 방향성**을 가지고 **반복적으로 가중치 값을 업데이트 하는 것.**
    - 오류 값 : 실제 값 - 예측 값
    - 분류의 실제 결괏값 = y, 피처 = x, 피처에 기반한 예측 함수 = F(x)
    - 오류식 **h(x) = y - F(x)**

In [2]:
# GBM 기반의 분류를 통해 사용자 행동 데이터 세트 예측 분류

import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
# 결정트리에서 사용한 함수들
# 중복된 피처명에 대해서 원본 피처명에 _1 또는 _2 를 추가로 부여해 새로운 피처명을 가지는 데이터프레임 반환

import pandas as pd
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df


# 데이터 프레임을 생성하는 함수
def get_human_dataset():
    
    # 각 데이터 파일은 공백으로 분리
    # read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('C:/Users/user/CUAI/DataSet/human_activity/features.txt',sep = '\s+',
                              header=None, names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df() 이용
    # 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    # 학습 피처, 테스트 피처 데이터 세트 로딩. 칼럼명은 feature_name
    x_train = pd.read_csv('C:/Users/user/CUAI/DataSet/human_activity/X_train.txt',sep='\s+',names=feature_name)
    x_test = pd.read_csv('C:/Users/user/CUAI/DataSet/human_activity/X_test.txt',sep='\s+',names=feature_name)
    
    # 학습 레이블, 테스트 레이블 데이터 세트 로딩. 칼럼명은 action
    y_train = pd.read_csv('C:/Users/user/CUAI/DataSet/human_activity/y_train.txt',sep='\s+',header=None, names=['action'])
    y_test = pd.read_csv('C:/Users/user/CUAI/DataSet/human_activity/y_test.txt',sep='\s+',header = None, names=['action'])
    
    # 로드된 학습/테스트용 데이터 프레임 모두 반환
    return x_train, x_test, y_train, y_test

In [5]:
# 시간이 너무 많이 걸려 주석처리함
#x_train, x_test, y_train, y_test = get_human_dataset()

# GBM 수행 시작 측정
#start_time = time.time()

#gb_clf = GradientBoostingClassifier(random_state = 10)
#gb_clf.fit(x_train, y_train)
#gb_pred = gb_clf.predict(x_test)
#gb_accuracy = accuravy_score(y_test, gb_pred)

#print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
#print('GBM 수행 시간 : {0:.1f}초'.format(time.time() - start_time))

## ✔ GBM 하이퍼 파라미터 및 튜닝


- 트리 기반 자체의 파라미터(max_depth, max_features)는 동일

- **<font color = 'Navy'>loss**
    - 경사 하강법에서 사용할 비용 함수 지정
    - 기본값은 deviance
    
- **<font color = 'Navy'>learning_rate**
    - GBM이 학습을 진행할 때마다 적용하는 학습률
    - 약한 분류기가 순차적으로 오류 값을 보정해 나가는 데 적용하는 계수
    - 기본값은 0.1
    - 작게 설정하면 업데이트 되는 값이 작아져서 최소 오류 값을 찾아 에측 성능이 높아질 가능성이 있다.
    - 작게 설정하면 모든 weak learner의 반복이 완료돼도 최소 오류 값을 찾지 못할 수 있음
    - 너무 크게 설정하면 최소 오류 값을 찾지 못하고 그냥 지나쳐 버려 예측 성능이 떨어질 가능성이 있으나 빠른 수행 가능
    - n_esitmators와 상호보완적으로 조합하여 사용
    
    
- **<font color = 'Navy'>n_estimator**
    - weak learner(약한 분류기) 개수
    - 많을수혹 예측 성능이 일정 수준까지는 좋아질 수 있으나 수행 시간이 오래 걸림
    - 기본값은 100
    
    
- **<font color = 'Navy'>subsample**
    - weak learner가 학습에 사용하는 데이터 샘플링 비율
    - 기본값은 1 : 전체 학습 데이터를 기반으로 학습한다는 의미
    - 과적합이 염러되는 경우 1보다 작은 값으로 설정

### GridSearchCV를 이용한 하이퍼 파라미터 최적화
- 시간이 너무 많이 걸려 넘어감. 교재 226p 참고

# <FONT COLOR = 'GREEN'>XGBoost(eXtra Gradient Boost) p.227~

- 트리 기반의 앙상블 학습에서 가장 각광받고 있는 알고리즘


- 자체적으로 교차 검증, 성능 평가, 피처 중요도 등의 ***시각화 기능***을 가지고 있다.


- 수행 속도를 향상시키기 위한 ***조기 중단 기능***이 있다.
    - n_estimators에 지정한 부스팅 반복 횟수에 도달하지 않더라도 예측 오류가 더 이상 개선되지 않으면 반복을 끝까지 수행하지 않는다,


- 장점
    - **뛰어난 예측 성능**
    - **GBM 대비 빠른 수행 시간**
    - **과적합 규제**
    - **Tree pruning(나무 가지치기)** : 더 이상 긍정 이득이 없는 분할을 가지치기하여 분할 수를 더 줄이는 장점
    - **자체 내장된 교차 검증**
    - **결손값 자체 처리**

In [2]:
import xgboost as xgb
from xgboost import XGBClassifier

In [4]:
print(xgb.__version__)

1.3.3


## ✔ 파이썬 래퍼 XGBoost  하이퍼 파라미터

### <font color = 'orange'>1. 일반 파라미터 

- 실행 시 선택을 위한 파라미터. 디폴트 파라미터 값을 바꾸는 경우는 거의 없음

- **<font color = 'Navy'>booster** : gbtree(tree based model) 또는 gblinear(linear model) 선택, 디폴트는 gbtree.


- **<font color = 'Navy'>silent** : 출력 메세지를 나타내고 싶지 않을 경우 1로 설정. 디폴트는 0.


- **<font color = 'Navy'>nthred** : cpu 실행 스레드 개수 조정. 디폴트는 cpu의 전체 스레드 사용

### <font color = 'orange'>2. 부스터 파라미터 

- 트리 최적화, 부스팅, regularization 등과 관련 파라미터 등을 지칭

- **<FONT COLOR = 'NAVY'>eta [default = 0.3, alias: learning rate]**
    - **GBM의 학습률과 같은 파라미터**
    - 0에서 1 사이의 값을 지정
    - 부스팅 스텝을 반복적으로 수행할 때 업데이트되는 학습률 값

    
- **<FONT COLOR = 'NAVY'>num_boost_rounds**
    - GBM의 n_estimators와 같은 파라미터
    

- **<FONT COLOR = 'NAVY'>min_child_weight[default = 1]**
    - **트리에서 추가적으로 가지를 나눌지를 결정하기 위해 필요한 데이터들의 weight 총합.**
    - 클수록 분할을 자제
    - 과적합을 조절하기 위해 사용
    
    
 
- **<FONT COLOR = 'NAVY'>gamma[default = 0, alias: min_split_loss]**
    - 트리의 리프 노드를 추가적으로 나눌지를 결정할 **최소 손실 감소 값**
    - 해당 값보다 큰 손실이 감소된 경우에 리프 노드를 분리
    - 값이 클수록 과적합 감소 효과
    
    
    
- **<FONT COLOR = 'NAVY'>max_depth[default = 6]**
    - 0을 지정하면 깊이에 제한이 없음
    - 높으면 특정 피처 조건에 특화되어 과적합 가능성이 높아지며 보통 3 ~ 10 사이의 값을 적용
    
    
    
    
    
- **<FONT COLOR = 'NAVY'>sub_sample[default = 1]**
    - **과적합을 제어**하기 위해 **데이터를 샘플링하는 비율 지정**
    - 0.5로 지정하면 전체 데이터의 절반을 트리를 생성하는 데 사용
    
    
    
- **<FONT COLOR = 'NAVY'>colsample_bytree[default = 1]**
    - GBM의 max_features와 유사
    - 트리 생성에 필요한 피처를 임의로 샘플링하는 데 사용
    - 피처가 많으면 과적합
    
    
    
- **<FONT COLOR = 'NAVY'>lambda[default = 1, alias: reg_lambda]**
    - L2 Regularization 적용 값
    - 피처 개수가 많을 경우 적용을 검토하며 **값이 클수록 과적합 감소 효과**가 있다.
    
    
    
- **<FONT COLOR = 'NAVY'>alpha[default = 0, alias: reg_alpha]**
    - L1 Regularization 적용 값
    - 피처 개수가 많을 경우 적용을 검토하며 **값이 클수록  과적합 감소 효과**가 있다.
    
    
    
- **<FONT COLOR = 'NAVY'>scale_pos_weight[default = 1]**
    - 특정 값으로 치우친 비대칭한 클래스로 구성된 경우 **데이터 균형을 유지하기 위한 파라미터**
    
  

### <font color = 'orange'>3. 학습 태스크 파라미터 

- 학습 수행 시의 객체 함수, 평가를 위한 지표 등을 설정하는 파라미터

- **<FONT COLOR = 'navy'>objective**
    - 최솟값을 가져야할 손실 함수 정의
    - **이진 분류 : binary:logistic**
    - **다중 분류 : multi:softmax**
        - 손실 함수가 multi:softmax인 경우에는 레이블 클래스의 개수인 num_class 파라미터 지정
    - multi:softprob : multi:softmax와 ㅇㅍ사하나 개별 레이블 클래스에 해당되는 예측 확률을 반환
    

    
- **<FONT COLOR = 'NAVY'>eval_metrix**
    - **검증에 사용되는 함수를 정의**
    - **rmse : Root Mean Square Error(회귀인 경우 기본값)**
    - mae : Mean Absolute Error
    - logloss : Negative log-likelihood
    - **error : Binary classification error rate(0.5 threshold) (분류인 경우 기본 값)**
    - merror : Multiclass classification error rate
    - mlogloss : Multiclass logloss
    - auc : Area under the curve

### 과적합 문제 해결 방안

- eta 값을 낮춘다(반대로 n_estimators나 num_round값은 높여준다).


- max_depth 값을 낮춘다.


- min_child_weight 값을 높인다.


- gamma 값을 높인다.


- subsample과 colsample_bytree 를 조정한다.